### 1. 뉴스레터 생성 GPT-3.5-Turbo

In [ ]:
import pandas as pd
import json
import time
from openai import OpenAI

In [ ]:
# GPT-3.5-Turbo 파인튜닝
class NewsLetterGenerator:
    def __init__(self, api_key, data_path, output_path="FineTune_data.jsonl"):
        self.api_key = api_key
        self.data_path = data_path
        self.output_path = output_path
        self.client = OpenAI(api_key=self.api_key)

    def load_data(self):
        return pd.read_csv(self.data_path, encoding='utf-8-sig')

    def create_FineTune_data(self, row):
        example = {
            "messages": [
                {
                    "role": "system",
                    "content": """당신은 초등학생들을 위해 원본 신문기사를 뉴스레터 형태로 바꿔서 작성해주는 기자입니다. 어려운 신문 기사를 뉴스레터 형태로 바꿔서 작성한다면, 학생들이 본문 내용을 더욱 쉽게 이해할 수 있을 것입니다. 아래의 [조건]들을 바탕으로, [원본 신문기사]를 뉴스레터 형태로 작성해주세요. 
                    조건 1 : 모든 문장에서 학생들에게 친밀감 있는 말투를 사용해주세요.
                    조건 2 : 적절한 이모지를 사용해주세요.
                    조건 3 : 결과물은 4개의 Part으로 구성해주세요. 구성은 다음과 같습니다.
                        1st Part : 간단한 인사와 함께 [원본 신문기사]의 주제에 대해서 소개해주세요. 
                        2nd Part : [원본 신문기사]에 등장하는 단어들 중, [어려운 어휘]에 속하는 단어들을 쉽게 풀어서 설명해주세요. 
                        3rd Part : [원본 신문기사]의 본문을 초등학생이 이해하기 쉬운 말로 풀어서 작성해주세요. 단, 본문의 내용이 지나치게 요약되면 안 됩니다. 본문 내용을 최대한 보존한 상태에서, 어려운 표현들만 쉽게 풀어서 작성해주세요.
                        4th Part : 본문 내용을 정리하면서 마무리 인사를 해주세요.
                    조건 4 : 글의 전반적인 구성과 형태를 뉴스레터처럼 구성해주세요.
                    """
                },
                {
                    "role": "user",
                    "content": '[원본 신문기사] : {}'.format(row["Input"])
                },
                {
                    "role": "user",
                    "content": '[어려운 어휘] : {}'.format(row["어려운 어휘"])
                },
                {
                    "role": "assistant",
                    "content": row["수정본"]
                }
            ]
        }
        return example

    def save_FineTune_data(self, tuning_data):
        with open(self.output_path, encoding="utf-8", mode="w") as file:
            for i in tuning_data:
                file.write(json.dumps(i) + "\n")
        print(f'Tuning data saved successfully as {self.output_path}')

    def upload_FineTune_data(self):
        with open(self.output_path, 'rb') as file:
            response = self.client.files.create(file=file, purpose="fine-tune")
        file_id = response.id
        print(f'File uploaded successfully with ID: {file_id}')
        return file_id

    def create_FineTune_model(self, file_id):
        tuning_job = self.client.fine_tuning.jobs.create(training_file=file_id, model="gpt-3.5-turbo")
        fine_tuning_job_id = tuning_job.id
        print(f"Fine-tuning job created successfully with ID : {fine_tuning_job_id}")
        return fine_tuning_job_id

    def check_FineTune_status(self, job_id):
        job = self.client.fine_tuning.jobs.retrieve(job_id)
        print(f"Job status: {job.status}")
        return job.status

    def get_FineTune_model_id(self, job_id):
        job = self.client.fine_tuning.jobs.retrieve(job_id)
        model_id = job.fine_tuned_model
        print(f"Created Fine-tuned model with ID : {model_id}")
        return model_id

In [ ]:
api_key = "사용자의_api_key를 입력"
data_path = '데이터_파일_이름'
generator = NewsLetterGenerator(api_key, data_path)

data = generator.load_data()

tuning_data = [generator.create_FineTune_data(row) for _, row in data.iterrows()]

generator.save_FineTune_data(tuning_data)

file_id = generator.upload_FineTune_data()

job_id = generator.create_FineTune_model(file_id)

generator.check_FineTune_status(job_id)

fine_tuned_model_id = generator.get_FineTune_model_id(job_id)

### 2. 내용일치 퀴즈 생성 GPT-3.5-Turbo

In [ ]:
api_key = "사용자의_api_key를 입력"
client = OpenAI(
    api_key=api_key
)

In [ ]:
# 퀴즈 생성을 위한 프롬프트
prompt = [
    {
        "role": "system",
        "content": """너는 어린이들을 위해 퀴즈를 생성하는 모델이야. [신문 기사]에 대해서 내용 일치 퀴즈를 생성해줘. 아래의 4가지 조건을 고려하여 퀴즈를 생성해줘.
        조건1. 1번부터 4번까지 4개의 statement를 출력해줘.
        조건2. 4개의 statement 중에서 3개의 statement는 true statement여야 해. True statement는 [신문 기사]의 내용과 일치하는 statement가 되어야 해.
        조건3. 4개의 statement 중에서 1개의 statement는 false statement여야 해. False statement는 [신문 기사]의 내용과 반대되는 statement가 되어야 해.
        조건4. True statement 뒤에는 (true), false statement 뒤에는 (false)로 표시를 해줘."""
    },
    {
         "role": "user",
        "content": '[신문기사] : {}'.format(article)
    }
]

In [ ]:
s_time = time.time()

# gpt-3.5-turbo에 쿼리 보내기
chat_completion = client.chat.completions.create(
    messages=prompt,
    model='gpt-3.5-turbo-1106')

e_time = time.time() - s_time
print(e_time)

In [ ]:
result = chat_completion.choices[0].message.content
print(result)

In [ ]:
# gpt-4-turbo에 쿼리 보내기
chat_completion_4 = client.chat.completions.create(
    messages=prompt,
    model='gpt-4-turbo-preview')

In [ ]:
result = chat_completion_4.choices[0].message.content
print(result)